# CSE 204 Lab 9: Decision Trees and Ensemble Methods

J.B. Scoggins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jbscoggi/teaching/blob/master/Polytechnique/CSE204/Lab9_answers.ipynb) 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jbscoggi/teaching/master?filepath=Polytechnique%2FCSE204%2FLab9_answers.ipynb)



## Introduction

In this lab you will learn how to use decision trees and ensemble methods to build a model that predicts the average price of houses for neighborhoods in the US state of California.  We will make heavy use of the following libraries to play with the dataset and build our models:

- [Pandas](https://pandas.pydata.org/) - python data analysis library
- [Scikit-learn](https://scikit-learn.org/stable/) - python machine learning library

Recall from the lecture that decision trees can be a powerful way to generate cheap and efficient models for classification and regression.  Some of the advantages of decision trees over other models include:

- They are easy to use, requiring little data preprocessing,
- Can be easily interpreted,
- Are useful for feature selection,
- Fast to build and evaluate,
- Non-linear.

On the other hand, some of the disadvantages include:

- Greedy tree building algorithms are not necessarily optimal (this is NP-complete),
- The number of samples is logarithmic in tree depth,
- Trees are unstable, meaning they can be easily perturbed with small differences in data,
- Decision trees are excellent overfitters,
- Since they only consider a single feature at a time, they have difficulty handling model additivity.

Often, it decision trees can be poor classifiers or regressors.  However, because of their fast training speed, they can be used to generate ensemble models, such as random forests, with boosting and bagging.  We will play with some of these concepts during this lab in order to compare to basic decision tree performance.  Before we get started, let's import and load the different packages we will use.



In [ ]:
# Import required packages
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor

import graphviz

# Setup pandas options
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format


## Introduction to Pandas

As we will be using the Pandas library, it is important you remember the core functionality from your previous labs.
If you want a quick Pandas refresher, you can follow a simple Pandas tutorial provided by the [Google Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/).  If you are not familiar with this course, you may find it useful to supplement your lectures and labs during your free time.  [Spend the next few minutes going through the tutorial, if needed.](https://colab.research.google.com/notebooks/mlcc/intro_to_pandas.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=pandas-colab&hl=en#scrollTo=TJffr5_Jwqvd)

## Step 1: Get to know your dataset

In this lab we will make use of the [California housing price dataset](https://developers.google.com/machine-learning/crash-course/california-housing-data-description).  This data was compiled from the 1990 census taken in California.  You can download the dataset with the Pandas code below.

In [ ]:
# Get the housing data
housing_data = pd.read_csv(
    "https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

# # Reindex and scale median house value
# housing_data = housing_data.reindex(
#     np.random.permutation(housing_data.index))

As you are probably well aware at this point, one of the most important aspects of machine learning and data science is to first understand your dataset.  Whenever you are introduced to a new dataset, it is crucial that learn everything you can about the data to help inform your model building strategy.  The Pandas library is very useful for this purpose.

**Exercise 1.1:** Using the describe() method to print information about the `housing_data` data frame and answer the following questions:

1. How many features are in the dataset?
2. How many samples are there?
3. Do mean and standard deviations make sense for what you expect each variable to be?
  - Are the latitude and longitude values consistent with California? (don't be afraid to check with Google maps)
  - What do you think the units of `total_rooms` and `total_bedrooms` are?
  - What about the `median_income`?
4. Which feature are we likely to want to try and predict?
5. Can you detect if there are any outliers in the data based on the minimum and maximum values?

In [ ]:
housing_data.describe()

# 9 features
# 17000 samples
# yes, rooms per block group, maybe in $10,000
# median_house_value
# The minimum values for total_(bed)rooms, population, and households seem very low and the max
# median_house_value is suspicious

**Exercise 1.2** From the previous question, it seems logical to think that the `total_rooms` and `total_bedrooms` are for an entire block, not a single house.  Let's check this assumption by creating two new variables called `rooms_per_person` and `bedrooms_per_person`.

In [ ]:
housing_data["rooms_per_person"] = housing_data["total_rooms"] / housing_data["population"]
housing_data["bedrooms_per_person"] = housing_data["total_bedrooms"] / housing_data["population"]
housing_data.describe()

# The maximum rooms still seems strange, like corrupted data

**Exercise 1.3:** Get a visual feel for the data by plotting the median house value on a scatter plot versus longitude and latitude (use color for house value).  Compare with a map of california such as [this one](https://www.google.com/search?q=california+maps&client=firefox-b-d&tbm=isch&source=iu&ictx=1&fir=KnETshNcnsi1VM%253A%252CMK2MjhZw7xRERM%252C_&vet=1&usg=AI4_-kSz1S_ut8rli9wcyp0A12LG1aVofg&sa=X&ved=2ahUKEwju59vrxsDhAhXqyIUKHZAkBBgQ9QEwBXoECAgQDg#imgrc=KnETshNcnsi1VM:).  (Hint: normalize the median house value by the maximum.)

- Does this make sense?
- Where are the most expensive homes?

In [ ]:
plt.scatter(
    housing_data['longitude'], housing_data['latitude'], 
    cmap="coolwarm",
    c=housing_data['median_house_value'] / housing_data['median_house_value'].max()
)

# Yes, looks like California
# San Francisco and Los Angeles

**Exercise 1.4:** Before building a model, let's build some expectation for what the important features are.  This will help us interpret our decision trees later on.  Use the `corr()` method from Pandas to build a correlation matrix.  Recall that a correlation matrix tells us how correlated any two features are.  For very positive values (close to 1.0), there is a strong correlation betwee the two features.  Likewise, for very negative values (close to -1.0), it means that the parameters are negatively correlated.  When the values are close to 0, there is no correlation.

If you want a visual representation of the correlation matrix, you can plot it using the `imshow()` method from matplotlib.

List which features (or groups of features) are strongly correlated (positive or negative).

In [ ]:
housing_data.corr()

# lat/lon
# total_(bed)rooms/households
# median_house_value/median_income
# (bed)rooms_per_person

**Exercise 1.5:** Use scatter plots to visualize the correlations between the groups you identified.

In [ ]:
# plt.scatter(housing_data['longitude'], housing_data['latitude'])

# plt.scatter(housing_data['total_rooms'], housing_data['total_bedrooms'])
# plt.scatter(housing_data['total_rooms'], housing_data['population'])
# plt.scatter(housing_data['total_rooms'], housing_data['households'])

plt.scatter(housing_data['median_income'], housing_data['median_house_value'])

**Exercise 1.6:** Notice anything strange about the scatter plot with `median_house_value`?  What's going here?  Use a histogram to get an idea of the distribution of this feature. 

- What does the histogram tell us about the distribution?
- What could this mean about how the data was collected or processed to build the database?

In [ ]:
housing_data['median_house_value'].hist(bins=20)

# There is a peak at 500000
# Housing values were clipped at $500,000

## Step 2: Train a simple decision tree to predict median house values

At this point we have pretty good idea about our dataset.  Let's try and create a simple model based on a decision tree.

**Exercise 2.1:** Split the dataset into a training and validation set.  Use the [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from Scikit-learn with `shuffle` set to `False`.  Check that the reduced datasets make sense with the `describe()` function.

In [ ]:
target = 'median_house_value'
X = housing_data.drop(target, axis=1)
y = housing_data[target]

training_features, testing_features, training_target, testing_target = train_test_split(
    X, y, test_size=0.3, shuffle=True)
training_features.describe()

**Exercise 2.2:** As a second check, redo the scatter plot from Exercise 1.3 for both the training and testing subsets.  

- Is this what you expected?  
- What went wrong?

This is an important lesson.  In all of our dataset checking from Exercise 1, we didn't get a feel for the ordering of the data.  Ideally, we want our validation and training data to be identically distributed, however we did not take into account that the dataset has a clear ordering.  Normally, the `train_test_split()` defaults to shuffle our data for us to avoid such issues but we have turned this off.  Go ahead and set `shuffle` to `True` above and rerun the cell and your scatter plots below to verify this fixes the problem.

In [ ]:
fig, axes = plt.subplots(1,2)
axes[0].scatter(
    training_features['longitude'], training_features['latitude'], 
    cmap="coolwarm",
    c=training_target / training_target.max()
)
axes[1].scatter(
    testing_features['longitude'], testing_features['latitude'], 
    cmap="coolwarm",
    c=testing_target / testing_target.max()
)
plt.show()

**Exercise 2.3:** Use the `DecisionTreeRegressor` from Scikit-learn to create a new decision tree model with a depth of 3 and train it on the training dataset using the `fit()` method.  You can find the API for DecisionTreeRegressor [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html).

In [ ]:
model = DecisionTreeRegressor(max_depth=10)
model.fit(training_features, training_target)

**Exercise 2.4:** Let's get a sense of how well the tree fits the data.  Use the tree's `predict` method on the testing features to get a prediction for the testing dataset.  Compute the mean squared error using the `mean_squared_error()` function from Scikit-learn.  Also, plot a scatter plot of the predicted median house values versus the testing target values. 

- Is the prediction good?
- What does the scatter plot look like?
- How many unique values are there in our predictions? Why is this?

In [ ]:
prediction = model.predict(testing_features)
print(mean_squared_error(prediction, testing_target))
plt.scatter(testing_target, prediction)

# Nope
# Several constant regions
# 8, this is 2^3, since we split the data into two groups at each split and we have 3 layers

**Exercise 2.5** You can visualize the trained decision tree with the following code.  As stated in the beginning, one advantage of such trees is that they are easily interpretible with graphs like this one.  

- What are the main features used in the tree?
- How do these compare to features you identified in Exercise 1.4?

In [ ]:
dot_data = export_graphviz(
    model, out_file=None, feature_names=list(training_features), filled=True, 
    rounded=True, special_characters=True)
graph = graphviz.Source(dot_data) 
graph 

**Exercise 2.6** Repeat 2.3 and 2.4 for different values of the maximum tree-depth.

- How does the fit improve with increasing depth?

Note that you can try visualizing the tree with larger depth values, but it becomes difficult as the tree grows.

## Step 3: Ensemble methods

We have seen that a single decision tree is not a very good estimator.  We can improve our predictions using a number of ensemble techniques such as bagging, random forests, and boosting.  Using Scikit-learn, it is easy to try all of these models.  Here are the main classes that we will test:

- [BaggingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor)
- [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
- [ExtraTreesRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html)
- [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html)

Follow each link to read about the details of each method.  You may also want to revisit the lecture notes on ensemble methods for a quick refresher of these terms.  In the following exercises you will train each type of regressor using a max_depth of 10 layers and test their prediction accuracy.  Finally, you can perform a hyperparameter study to try and find the optimal model/parameter pair which minimizes the MSE on the testing dataset.

**Exercise 3.1:** Create and fit a bagging regressor based on decision trees with a max depth of 10 using 100 trees, and maximum samples and features of 50%.  Compute the MSE on the testing dataset and compare the true median house values with the predictions using a scatter plot.

In [ ]:
bagging = BaggingRegressor(
    DecisionTreeRegressor(max_depth=10), 
    n_estimators=100, max_samples=0.5, max_features=0.5
)
bagging.fit(training_features, training_target)
bagging_prediction = bagging.predict(testing_features)
print(mean_squared_error(bagging_prediction, testing_target))
plt.scatter(testing_target, bagging_prediction)

**Exercise 3.2:** Create and fit a random forest regressor with a max depth of 10.  Compute the MSE on the testing dataset and compare the true median house values with the predictions using a scatter plot.

In [ ]:
random_forest = RandomForestRegressor(max_depth=10)
random_forest.fit(training_features, training_target)
rf_prediction = random_forest.predict(testing_features)
print(mean_squared_error(rf_prediction, testing_target))
plt.scatter(testing_target, rf_prediction)

**Exercise 3.3:** Create and fit an extremely randomized trees regressor (ExtraTreesRegressor) with a max depth of 10.  Compute the MSE on the testing dataset and compare the true median house values with the predictions using a scatter plot.

In [ ]:
randomized_trees = ExtraTreesRegressor(max_depth=10)
randomized_trees.fit(training_features, training_target)
rt_prediction = randomized_trees.predict(testing_features)
print(mean_squared_error(rt_prediction, testing_target))
plt.scatter(testing_target, rt_prediction)

**Exercise 3.4:** Create and fit a tree ensemble regressor with boosting (AdaBoostRegressor) with a max depth of 10.  Compute the MSE on the testing dataset and compare the true median house values with the predictions using a scatter plot.

In [ ]:
boosted = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=10), n_estimators=10)
boosted.fit(training_features, training_target)
boosted_prediction = boosted.predict(testing_features)
print(mean_squared_error(boosted_prediction, testing_target))
plt.scatter(testing_target, boosted_prediction)

**Exercise 3.5:** Plot the MSE for each of the four ensemble methods above versus the maximum depth used (from 2 to 20).

- At what maximum depth do the different methods more or less converge?
- Which is the best method in terms of minimum MSE obtained?

In [ ]:
def test_error(model):
    model.fit(training_features, training_target)
    prediction = model.predict(testing_features)
    return mean_squared_error(prediction, testing_target)


min_depth = 2
max_depth = 20

estimators = 100

mse = np.zeros((max_depth-min_depth+1, 4))
for i,D in enumerate(range(2,max_depth+1)):
    print(D)
    model = BaggingRegressor(
        DecisionTreeRegressor(max_depth=D), 
        n_estimators=estimators, max_samples=0.5, max_features=0.5
    )
    mse[i,0] = test_error(model)
    
    model = RandomForestRegressor(max_depth=D, n_estimators=estimators)
    mse[i,1] = test_error(model)
    
    model = ExtraTreesRegressor(max_depth=D, n_estimators=estimators)
    mse[i,2] = test_error(model)
    
    model = AdaBoostRegressor(
        DecisionTreeRegressor(max_depth=D), n_estimators=estimators)
    mse[i,3] = test_error(model)

In [ ]:
plt.plot(range(min_depth, max_depth+1), mse[:,0])
plt.plot(range(min_depth, max_depth+1), mse[:,1])
plt.plot(range(min_depth, max_depth+1), mse[:,2])
plt.plot(range(min_depth, max_depth+1), mse[:,3])
plt.legend(['Bagging', 'Random Forest', 'Extremely Random Tree', 'Boosted'])
plt.show()

## Conclusions

That's it for this lab!  Note there are still several ways you can try and improve the regression.  It might even be worth trying to use an entirely different model such as a neural network to see if you can obtain better performance.  Over time, you will build up an intuition for which models might work best on different datasets.  

If you want more practice with trees and ensemble methods, checkout the Iris dataset below which is a classification problem.  You can use everything you learned here by replacing "Regressor" with "Classifier".  Happy coding!

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()